In [1]:
from datascience import *
import numpy as np, pandas as pd, matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
pandas_table = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines= True)
articles = Table.from_df(pandas_table)
articles = articles.with_column('length', articles.apply(len, 'headline'))

In [3]:
sarcastic = articles.where('is_sarcastic', 1)
real = articles.where('is_sarcastic', 0)

In [4]:
sarcastic_headlines = sarcastic.select('headline')
real_headlines = real.select('headline')

In [5]:
useless_words = ['a', 'at', 'and', 'be', 'by', 'for', 'is', 'in', 'it', 'of', 'on', 'to', 'the', 'of', '--']

In [6]:
sarcastic_words = []

for i in sarcastic_headlines.column(0):
    for j in i.split():
        sarcastic_words.append(j)

sarcastic_tbl = Table().with_column('words', sarcastic_words).where('words', are.not_contained_in(useless_words))
sarcastic_tbl = sarcastic_tbl.group('words').sort(1, descending=True)
sarcastic_words = sarcastic_tbl.column(0)
sarcastic_tbl

words,count
man,1143
with,1008
new,973
from,664
out,559
after,546
up,507
he,499
area,487
about,483


In [7]:
real_words = []

for i in real_headlines.column(0):
    for j in i.split():
        real_words.append(j)

real_tbl = Table().with_column('words', real_words)
real_tbl = real_tbl.group('words').where('words', are.not_contained_in(useless_words)).sort(1, descending=True)
real_tbl

words,count
trump,957
with,943
you,732
this,690
new,664
from,661
about,644
how,639
your,575
are,555


In [8]:
sarcastic_unique = sarcastic_tbl.where(0, are.not_contained_in(real_tbl.column(0)))
sarcastic_words = sarcastic_unique.column(0)
sarcastic_unique

words,count
fucking,100
shit,66
clearly,56
fuck,45
realizes,42
unable,37
kavanaugh,36
archives:,32
slowly,30
per,30


In [9]:
real_unique = real_tbl.where(0, are.not_contained_in(sarcastic_tbl.column(0)))
real_words = real_unique.column(0)
real_unique

words,count
huffpost,58
email:,57
queer,49
watch:,48
trans,48
kardashian,47
jenner,40
lgbt,39
'snl',34
lgbtq,32


In [9]:
def unique_count(headline):
    '''Returns the number of times certain words are in a headline'''
    count_sarcastic = 0
    count_real = 0
    for i in headline.split():
        count_sarcastic += np.count_nonzero(sarcastic_words == i)
        #count_real += np.count_nonzero(real_words == i)
        
    #return {'sarcastic':count_sarcastic, 'real':count_real}
    return count_sarcastic

In [10]:
#sarcastic_counts = articles.apply(lambda x: unique_count(x)['sarcastic'], 'headline')
#real_counts = articles.apply(lambda x: unique_count(x)['real'], 'headline')
sarcastic_counts = articles.apply(unique_count, 'headline')

In [12]:
articles_with_counts = articles.with_columns('Sarcastic Words', sarcastic_counts)

In [13]:
articles_with_counts.sort('Sarcastic Words', descending=True).show(5)

article_link,headline,is_sarcastic,length,Sarcastic Words
https://www.theonion.com/hot-wheels-ranked-number-one-to ...,"hot wheels ranked number one toy for rolling down ramp, ...",1,926,39
https://entertainment.theonion.com/12-years-a-slave-capt ...,"'12 years a slave,' 'captain phillips,' 'american hustle ...",1,238,15
https://www.theonion.com/maya-angelou-poet-author-civil- ...,"maya angelou, poet, author, civil rights activist, and—h ...",1,254,14
https://entertainment.theonion.com/mia-farrow-it-s-possi ...,mia farrow: 'it's possible my son was fathered by frank ...,1,199,13
https://www.theonion.com/elmore-leonard-modern-prose-mas ...,"elmore leonard, modern prose master, noted for his terse ...",1,237,11
